In [ ]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, RandomFlip, RandomRotation
from tensorflow.keras.metrics import Precision, Recall, AUC


In [ ]:
# To import datasets from google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip -o gdrive/MyDrive/BU/BreaKHis_v1.zip

In [ ]:
import tensorflow as tf
image_size = (700, 460)
batch_size = 32
# Code from https://keras.io/examples/vision/image_classification_from_scratch/
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "BreaKHis_v1/BreaKHis_v1/histology_slides/data",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "BreaKHis_v1/BreaKHis_v1/histology_slides/data",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)


In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Rescaling, RandomFlip, RandomRotation
from tensorflow.keras.models import Sequential

from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=10,
    min_lr=0.00001
)
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)

# Attention layer code adapted from https://www.kaggle.com/code/haithemhermessi/attention-mechanism-keras-as-simple-as-possible

class AttentionLayer(layers.Layer):
    def __init__(self):
        super(AttentionLayer, self).__init__()

    def build(self, input_shape):
        self.attention_weights = self.add_weight(
            shape=(input_shape[-1], 1),
            initializer="random_normal",
            trainable=True,
        )

    def call(self, inputs):
        attention_score = tf.matmul(inputs, self.attention_weights)
        attention_weights = tf.nn.softmax(attention_score, axis=1)
        weighted_output = inputs * attention_weights
        return tf.reduce_sum(weighted_output, axis=1)

input_shape = (700, 460, 3)
model_with_attention = Sequential(
    [
        keras.Input(shape=input_shape),
        Rescaling(scale=1./255),


        Conv2D(32, kernel_size=(3,3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(64, kernel_size=(3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),

        Conv2D(128, kernel_size=(3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),

        Conv2D(256, kernel_size=(3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),


        Flatten(),
        Dropout(0.5),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(64, activation="relu"),
        Dense(1, activation="sigmoid"),
    ]
)



In [ ]:
model_with_attention.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
history = model_with_attention.fit(
    train_ds,
    epochs=40,
    validation_data=val_ds,
    callbacks=[lr_scheduler, early_stopping]
)


In [ ]:
final_val_accuracy = history.history['val_accuracy'][-1]
print(final_val_accuracy)